In [12]:
%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb
#run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
warnings.filterwarnings('ignore')

In [14]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [15]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [16]:
tabla_pob_ocupada_asalariada = calcular_tabla_poblacion_ocupada_asalariada(df_people_pool, tipo='Urbano', base='Individual')

In [17]:

tasa, size, ratio, error = ratio_tasa_inactividad(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
ratio_inactividad = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = ratio_tasas_empleo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
empleo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = ratio_tasas_desempleo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
desempleo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = acceso_aguinaldo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_aguinaldo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = acceso_obra_social(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_obra_social = pd.concat([tasa, size, ratio, error], axis=1)

In [18]:

tasa, size, ratio, error = trabajo_formal(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_descuento_jubilatorio = pd.concat([tasa, size, ratio, error], axis=1)

In [19]:
tasa, size, ratio, error = horas_mediana_remuneradas(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
horas_remuneradas_mediana = pd.concat([tasa, size, ratio, error], axis=1)

In [23]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)


In [24]:
empleo_asalariado_con_aguinaldo.index = empleo_asalariado_con_aguinaldo.index.map(dict_cod_provincia)
empleo_asalariado_con_descuento_jubilatorio.index = empleo_asalariado_con_descuento_jubilatorio.index.map(dict_cod_provincia)
empleo_asalariado_con_obra_social.index = empleo_asalariado_con_obra_social.index.map(dict_cod_provincia)


In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

In [30]:
# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

In [34]:
# Create bar traces
bar_trace1 = go.Bar(
    x=empleo_asalariado_con_descuento_jubilatorio.index,
    y=empleo_asalariado_con_descuento_jubilatorio['Formalidad'],
    name='Desc jubilatorio',
    marker_color=fundar_colores_primarios[1]
)

bar_trace2 = go.Bar(
    x=empleo_asalariado_con_aguinaldo.index,
    y=empleo_asalariado_con_aguinaldo['Acceso a aguinaldo'],
    name='Aguinaldo',
    marker_color=fundar_colores_primarios[0]
)

bar_trace3 = go.Bar(
    x=empleo_asalariado_con_obra_social.index,
    y=empleo_asalariado_con_obra_social['Acceso a OS'],
    name='Obra Social',
    marker_color=fundar_colores_primarios[2]
)

# Create a layout
layout = go.Layout(
    title='Formalidad',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    font_size = 20
)

# Create a Figure
fig = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3], layout=layout)

fig.add_shape(
    type='line',
    x0=empleo_asalariado_con_obra_social.index[0],
    y0=100,
    x1=empleo_asalariado_con_obra_social.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)
#fig.update_yaxes(range=[0, 200])

# Save the plot as an HTML file
file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'
filename = 'indicador_formalidad_chequeo.html'
plot(fig, filename=file_path + filename, auto_open=False)


'/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/indicador_formalidad_chequeo.html'

In [43]:
descuento_jubilatorio = empleo_asalariado_con_descuento_jubilatorio['Formalidad'].tolist()
indices = empleo_asalariado_con_descuento_jubilatorio.index.tolist()
indicadores = list(zip(indices, descuento_jubilatorio))
indicadores_ordenados = sorted(indicadores, key=lambda x: x[1], reverse=True)
indices_ordenados = [x[0] for x in indicadores_ordenados]
descuento_jubilatorio_ordenado = [x[1] for x in indicadores_ordenados]
bar_trace1 = go.Bar(
    x=indices_ordenados,
    y=descuento_jubilatorio_ordenado,
    name='Desc jubilatorio',
    marker_color=fundar_colores_primarios[1]
)
bar_trace2 = go.Bar(
    x=empleo_asalariado_con_aguinaldo.index,
    y=empleo_asalariado_con_aguinaldo['Acceso a aguinaldo'],
    name='Aguinaldo',
    marker_color=fundar_colores_primarios[0]
)

bar_trace3 = go.Bar(
    x=empleo_asalariado_con_obra_social.index,
    y=empleo_asalariado_con_obra_social['Acceso a OS'],
    name='Obra Social',
    marker_color=fundar_colores_primarios[2]
)

# Create a layout
layout = go.Layout(
    title='Formalidad',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    font_size = 20,
    legend=dict(y=1.1, x=0.7, orientation='h'),

)

fig.add_shape(
    type='line',
    x0=indices_ordenados[0],
    y0=100,
    x1=indices_ordenados[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)
# Create a Figure
fig = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3], layout=layout)
# Save the plot as an HTML file
file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'
filename = 'indicador_formalidad_chequeo_sorted.html'
plot(fig, filename=file_path + filename, auto_open=False)


'/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/indicador_formalidad_chequeo_sorted.html'